In [2]:
import torch

In [7]:
x = torch.randn(6000,1,224,224)
card =x.numel()
x = torch.where(x>=2.7,x,0)
print(x.shape)
x = x.to_sparse()
x



torch.Size([6000, 1, 224, 224])


tensor(indices=tensor([[   0,    0,    0,  ..., 5999, 5999, 5999],
                       [   0,    0,    0,  ...,    0,    0,    0],
                       [   2,    5,    5,  ...,  220,  222,  222],
                       [  31,   68,  185,  ...,   19,   35,  121]]),
       values=tensor([2.8037, 2.8234, 3.1526,  ..., 2.8346, 3.6942, 2.7745]),
       size=(6000, 1, 224, 224), nnz=1043903, layout=torch.sparse_coo)

In [16]:
(1-x._nnz()/card)*100

99.65306056016156

In [66]:
x.indices().shape

torch.Size([4, 30108023])

In [6]:
import torch
import torch.nn.functional as F


indices = torch.tensor([
    [0, 0, 0, 0], [0, 1, 2, 2], [0, 2, 3, 3]

])
values = torch.tensor([
    1.0, 2.0, 3.0

])
dense_shape = (1200, 1, 448, 448)


sparse_tensor = torch.sparse.FloatTensor(indices.t(), values, torch.Size(dense_shape))


In [43]:
test = torch.zeros(4,1)
test_add =torch.tensor([1,1,1,1])
test_add.shape


torch.Size([4])

In [100]:
test.shape
for i in range(100):
    test = torch.cat((test,test_add.unsqueeze(1)),dim=1)
test.shape

torch.Size([4, 344972])

In [8]:

import torch
import torch.nn.functional as F


indices = x.indices()
print(indices.shape)
values = x.values()
print(values.shape)
dense_shape = [shape for shape in x.size()]
print("dense_shape",dense_shape)

indices =indices.t()
print(dense_shape)
kernel = torch.tensor([
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0]
]).reshape(1, 1, 3, 3)  

convolution_sum = None


global_indices = None
global_values = None


chunk_size = 100
num_chunks = dense_shape[0] // chunk_size
print(num_chunks)
from tqdm import tqdm

for i in tqdm(range(num_chunks)):
   
    chunk_indices = indices[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]
 
    chunk_indices[:, 0] -= i * chunk_size  
    chunk_values = values[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]

    chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([chunk_size, 1, 224, 224]))


    chunk_dense_tensor = chunk_sparse_tensor.to_dense()

    conv_output = F.conv2d(chunk_dense_tensor, kernel, stride=1, padding=1)

    conv_sum = conv_output.sum(dim=0)

   
    if convolution_sum is None:
        convolution_sum = conv_sum
    else:
        convolution_sum += conv_sum


  
    conv_output = conv_output.to_sparse()
    add_indices=(conv_output.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]).to(conv_output.indices().device))
    if global_indices is None:
        global_indices= add_indices
    else : 
        global_indices = torch.cat((global_indices,add_indices),dim =1)
    #global_indices.append(conv_output.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]))#.to(conv_output_sparse.indices().device))
    if global_values is None: 
        global_values = conv_output.values()
        print(global_values.shape)
    else: 
        global_values = torch.cat((global_values,conv_output.values()),dim=0)    
    


global_sparse_tensor = torch.sparse.FloatTensor(global_indices, global_values, torch.Size(dense_shape))


print("Somme des convolutions sur la dimension 0:")
print(convolution_sum)

print("\nTenseur sparse global:")
print(global_sparse_tensor)


torch.Size([4, 1043903])
torch.Size([1043903])
dense_shape [6000, 1, 224, 224]
[6000, 1, 224, 224]
60


  7%|▋         | 4/60 [00:00<00:01, 39.03it/s]

torch.Size([102719])


100%|██████████| 60/60 [00:02<00:00, 28.64it/s]


Somme des convolutions sur la dimension 0:
tensor([[[-1.1610e+02,  2.1572e+01, -5.5384e+01,  ..., -4.0113e+01,
          -2.0901e+01,  1.4180e+02],
         [-1.8457e+02,  3.0984e+01, -3.3409e+01,  ..., -4.6547e+01,
           7.8012e-02,  2.0706e+02],
         [-2.0818e+02,  1.7288e+01,  1.3909e+01,  ..., -4.0940e+01,
          -1.2262e+01,  2.1060e+02],
         ...,
         [-1.6993e+02,  5.2541e+01, -2.6657e+01,  ..., -4.7880e+01,
           3.0570e+01,  1.9438e+02],
         [-1.7162e+02,  1.4624e+01, -5.8739e+01,  ..., -1.8731e+01,
           2.7954e+01,  1.8683e+02],
         [-1.0731e+02,  3.5705e+01, -4.8175e+01,  ..., -6.6451e+00,
           2.4412e+00,  1.1077e+02]]])

Tenseur sparse global:
tensor(indices=tensor([[   0,    0,    0,  ..., 5999, 5999, 5999],
                       [   0,    0,    0,  ...,    0,    0,    0],
                       [   1,    1,    2,  ...,  223,  223,  223],
                       [  30,   32,   30,  ...,   36,  120,  122]]),
       values=ten

In [80]:
indices.shape

torch.Size([4, 30106187])

In [ ]:


sparse_tensor = torch.sparse.FloatTensor(indices.t(), values, torch.Size(dense_shape))

kernel = torch.tensor([
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0],
    [1.0, 0.0, -1.0]
]).reshape(1, 1, 3, 3)  

convolution_sum = None


global_indices = []
global_values = []


chunk_size = 100
num_chunks = dense_shape[0] // chunk_size

for i in range(num_chunks):
   
    chunk_indices = indices[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]
    chunk_indices[:, 0] -= i * chunk_size  
    chunk_values = values[(indices[:, 0] >= i * chunk_size) & (indices[:, 0] < (i + 1) * chunk_size)]

    chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([chunk_size, 1, 224, 224]))


    chunk_dense_tensor = chunk_sparse_tensor.to_dense()

    conv_output = F.conv2d(chunk_dense_tensor, kernel, stride=1, padding=1)

    conv_sum = conv_output.sum(dim=0)

   
    if convolution_sum is None:
        convolution_sum = conv_sum
    else:
        convolution_sum += conv_sum

  
    conv_output_sparse = conv_output.to_sparse()
    global_indices.append(conv_output_sparse.indices() + torch.tensor([[i * chunk_size], [0], [0], [0]]).to(conv_output_sparse.indices().device))
    global_values.append(conv_output_sparse.values())

global_indices = torch.cat(global_indices, dim=1)
global_values = torch.cat(global_values)
global_sparse_tensor = torch.sparse.FloatTensor(global_indices, global_values, torch.Size(dense_shape))


print("Somme des convolutions sur la dimension 0:")
print(convolution_sum)

print("\nTenseur sparse global:")
print(global_sparse_tensor)


In [30]:
print(convolution_sum.shape)

torch.Size([1, 224, 224])


In [5]:
dense =x.to_dense()


conv_output = F.conv2d(dense, kernel, stride=1, padding=1)
del dense
conv_output = conv_output.to_sparse()
conv_output

tensor(indices=tensor([[0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2],
                       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                       [0, 1, 1, 1, 2, 2, 3, 3, 2, 2, 3, 3, 4, 4],
                       [1, 1, 1, 3, 1, 3, 1, 3, 2, 4, 2, 4, 2, 4]]),
       values=tensor([ 1.,  1., -2.,  2., -2.,  2., -2.,  2., -3.,  3., -3.,
                       3., -3.,  3.]),
       size=(12000, 1, 224, 224), nnz=14, layout=torch.sparse_coo)

In [2]:
!pip install onnx
import onnx

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 88.2 MB/s eta 0:00:00m eta 0:00:010:0101
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.2/309.2 KB 44.6 MB/s eta 0:00:00


In [5]:
!pip install onnx2pytorch

from onnx2pytorch import ConvertModel
onnx_model = onnx.load('vgg19-7.onnx')



Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 KB 1.2 MB/s eta 0:00:00


In [6]:
# Convertir le modèle ONNX en modèle PyTorch
pytorch_model = ConvertModel(onnx_model)


Automatic inference of operator: dropout
Automatic inference of operator: dropout


In [26]:
list = []
for param in pytorch_model.named_parameters():
    list.append(param[0])

In [27]:
list

['Conv_vgg0_conv0_fwd.weight',
 'Conv_vgg0_conv0_fwd.bias',
 'Conv_vgg0_conv1_fwd.weight',
 'Conv_vgg0_conv1_fwd.bias',
 'Conv_vgg0_conv2_fwd.weight',
 'Conv_vgg0_conv2_fwd.bias',
 'Conv_vgg0_conv3_fwd.weight',
 'Conv_vgg0_conv3_fwd.bias',
 'Conv_vgg0_conv4_fwd.weight',
 'Conv_vgg0_conv4_fwd.bias',
 'Conv_vgg0_conv5_fwd.weight',
 'Conv_vgg0_conv5_fwd.bias',
 'Conv_vgg0_conv6_fwd.weight',
 'Conv_vgg0_conv6_fwd.bias',
 'Conv_vgg0_conv7_fwd.weight',
 'Conv_vgg0_conv7_fwd.bias',
 'Conv_vgg0_conv8_fwd.weight',
 'Conv_vgg0_conv8_fwd.bias',
 'Conv_vgg0_conv9_fwd.weight',
 'Conv_vgg0_conv9_fwd.bias',
 'Conv_vgg0_conv10_fwd.weight',
 'Conv_vgg0_conv10_fwd.bias',
 'Conv_vgg0_conv11_fwd.weight',
 'Conv_vgg0_conv11_fwd.bias',
 'Conv_vgg0_conv12_fwd.weight',
 'Conv_vgg0_conv12_fwd.bias',
 'Conv_vgg0_conv13_fwd.weight',
 'Conv_vgg0_conv13_fwd.bias',
 'Conv_vgg0_conv14_fwd.weight',
 'Conv_vgg0_conv14_fwd.bias',
 'Conv_vgg0_conv15_fwd.weight',
 'Conv_vgg0_conv15_fwd.bias',
 'Gemm_vgg0_dense0_fwd.weigh

In [20]:
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
image = Image.open('voilier.jpeg')
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
   
    
])

image = transform(image)
print(image.shape)
pytorch_model=pytorch_model.eval()
pytorch_model(image.unsqueeze(0))

torch.Size([3, 224, 224])


TypeError: dropout() missing 2 required positional argument: "p", "train"

In [3]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src')
sys.path.append('../util')
import torch
import torch.nn as nn
import numpy as np

In [4]:
from sparse import SparseEvaluation
x = torch.randn(600,64,224,224)
card =x.numel()
x = torch.where(x>=4,x,0)
print(x.shape)
x = x.to_sparse()
x



torch.Size([600, 64, 224, 224])


tensor(indices=tensor([[  0,   0,   0,  ..., 599, 599, 599],
                       [  0,   1,   2,  ...,  61,  61,  62],
                       [147, 221,  46,  ...,  65,  96, 178],
                       [ 57,  24, 100,  ..., 183,  14, 106]]),
       values=tensor([4.1743, 4.8182, 4.3344,  ..., 4.1460, 4.1132, 4.3921]),
       size=(600, 64, 224, 224), nnz=61182, layout=torch.sparse_coo)

In [5]:
with torch.no_grad():
    conv = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3)
function = lambda x: x


In [6]:
import timeit
test = SparseEvaluation(x,function=function,chunck_size=10)
with torch.no_grad():
    test.evaluate_all_chuncks()

[600, 64, 224, 224]
Sparse evaluation, num_chunks 60
1
torch.Size([4, 61182])
torch.Size([61182])
dense_shape [600, 64, 224, 224]
[600, 64, 224, 224]


  0%|          | 0/60 [00:00<?, ?it/s]/home/guillaume/Documents/AbstractTorch/AbstractTorch/notebook/../src/sparse.py:59: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:618.)
  chunk_sparse_tensor = torch.sparse.FloatTensor(chunk_indices.t(), chunk_values, torch.Size([self.chunk_size, self.dense_shape[1],self.dense_shape[2], self.dense_shape[3]]))
  8%|▊         | 5/60 [00:00<00:01, 41.38it/s]

torch.Size([1027])


100%|██████████| 60/60 [00:01<00:00, 39.22it/s]

Somme des convolutions sur la dimension 0:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.

In [49]:
import torch
device = torch.device("cpu")
dim_H = 55
dim_W = 55
num_chanel = 3
def create_sparse_tensor(alpha, dims=(num_chanel*dim_H*dim_W, num_chanel, dim_W, dim_H)):
    # Liste pour stocker les indices des éléments non nuls
    indices = []
    # Liste pour stocker les valeurs des éléments non nuls
    values = []

    # Nombre d'éléments non nuls
    num_elements = dims[0]

    # Boucle pour créer les indices et les valeurs
    for i in range(num_elements):
        # Calcul des indices pour les dimensions (3, 128, 128)
        dim_3 = i // (dim_W * dim_H)
        rem = i % (dim_W * dim_H)
        dim_1 = rem // dim_W
        dim_2 = rem % dim_H

        # Indices des éléments non nuls
        indices.append([i, dim_3, dim_1, dim_2])
        # Valeurs des éléments non nuls
        values.append(alpha)

    # Conversion des listes en tenseurs PyTorch
    indices_tensor = torch.tensor(indices, dtype=torch.int32).t()
    values_tensor = torch.tensor(values, dtype=torch.float)

    # Création du tenseur épars
    sparse_tensor = torch.sparse_coo_tensor(indices_tensor, values_tensor, size=dims)

    return sparse_tensor

# Exemple d'utilisation
alpha = 0.00001
sparse_tensor = create_sparse_tensor(alpha).coalesce().to(device)
print(sparse_tensor)


tensor(indices=tensor([[   0,    1,    2,  ..., 9072, 9073, 9074],
                       [   0,    0,    0,  ...,    2,    2,    2],
                       [   0,    0,    0,  ...,   54,   54,   54],
                       [   0,    1,    2,  ...,   52,   53,   54]]),
       values=tensor([1.0000e-05, 1.0000e-05, 1.0000e-05,  ...,
                      1.0000e-05, 1.0000e-05, 1.0000e-05]),
       size=(9075, 3, 55, 55), nnz=9075, layout=torch.sparse_coo)


In [50]:
import torch
import torch.nn.functional as F
from torch.sparse import FloatTensor
from typing import Callable
device = torch.device("cpu")

class SparseEvaluation:
    def __init__(self, x: FloatTensor, chunk_size: int, mask_coef : FloatTensor = None , function: Callable = None):
        self.x = x
        if function is None : 
            self.function = lambda x: x 
        else: 
            self.function = function
        self.chunk_size = chunk_size
        self.dense_shape = list(x.size())
        x0 = torch.zeros(1, self.dense_shape[1], self.dense_shape[2], self.dense_shape[3]).to(device)
        if mask_coef is None: 
            self.mask_coef = torch.ones_like(x0)
        else: self.mask_coef = mask_coef
        

        # Nombre de chunks
        self.num_chunks = (self.dense_shape[0] + self.chunk_size - 1) // self.chunk_size

        # Taille de sortie de la fonction
        
        self.output_size = list(self.function(x0).shape)
        self.output_size[0] = self.dense_shape[0]

        self.indices = self.x.indices()
        self.values = self.x.values()

        self.indices = self.indices.t()
        self.function_sum = None
        self.global_storage = {
            'indices': [],
            'values': []
        }

    def evaluate_all_chunks(self):
        with torch.no_grad():
            from tqdm import tqdm
            for i in tqdm(range(self.num_chunks)):
                chunk_start = i * self.chunk_size
                chunk_end = min((i + 1) * self.chunk_size, self.dense_shape[0])
                mask = (self.indices[:, 0] >= chunk_start) & (self.indices[:, 0] < chunk_end)

                chunk_indices = self.indices[mask]
                chunk_indices[:, 0] -= chunk_start

                chunk_values = self.values[mask]
                chunk_size = chunk_end - chunk_start

                chunk_sparse_tensor = torch.sparse.FloatTensor(
                    chunk_indices.t(), chunk_values,
                    torch.Size([chunk_size] + self.dense_shape[1:])
                )

                chunk_dense_tensor = chunk_sparse_tensor.to_dense().to(device)
                func_output = self.function(self.mask_coef *chunk_dense_tensor)

           
                func_sum = torch.abs(func_output).sum(dim=0)
                if self.function_sum is None:
                    self.function_sum = func_sum
                else:
                    self.function_sum += func_sum

             
                func_output_sparse = func_output.to_sparse()
                add_indices = func_output_sparse.indices().to(torch.int32) + torch.tensor(
                    [[chunk_start], [0], [0], [0]], dtype=torch.int32, device=device
                )

                self.global_storage['indices'].append(add_indices.cpu())
                self.global_storage['values'].append(func_output_sparse.values().cpu())


                del chunk_dense_tensor, chunk_sparse_tensor, func_output, func_output_sparse, add_indices
                torch.cuda.empty_cache()

            # Concaténation finale des indices et valeurs
            global_indices = torch.cat(self.global_storage['indices'], dim=1)
            global_values = torch.cat(self.global_storage['values'], dim=0)

            global_sparse_tensor = torch.sparse_coo_tensor(global_indices, global_values, size=self.output_size)
            #global_sparse_tensor = torch.sparse.FloatTensor(
             #   global_indices, global_values, self.output_size
            #)

            print("Somme des convolutions sur la dimension 0:")
            print(self.function_sum)

            print("\nTenseur sparse global:")
            return global_sparse_tensor,self.function_sum


In [51]:
import torch
import torch.nn as nn
import numpy as np

with torch.no_grad():
    conv = nn.Conv2d(in_channels=3, out_channels=64,kernel_size=3).to(device)
    conv.bias.data = torch.zeros_like(conv.bias.data)
    conv2 = nn.Conv2d(in_channels=64, out_channels=128,kernel_size=3).to(device)
    conv2.bias.data = torch.zeros_like(conv2.bias.data)
function = lambda x: conv(x)
function2 = lambda x: conv2(x)

In [63]:
import timeit
test = SparseEvaluation(sparse_tensor,function=function,chunk_size=200)
with torch.no_grad():
    result,sum = test.evaluate_all_chunks()
    result = result.coalesce()

    result_2 = function(sparse_tensor.to_dense())

  0%|          | 0/46 [00:00<?, ?it/s]

100%|██████████| 46/46 [00:01<00:00, 27.53it/s]


Somme des convolutions sur la dimension 0:
tensor([[[2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05],
         [2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05],
         [2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05],
         ...,
         [2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05],
         [2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05],
         [2.4665e-05, 2.4665e-05, 2.4665e-05,  ..., 2.4665e-05,
          2.4665e-05, 2.4665e-05]],

        [[2.5257e-05, 2.5257e-05, 2.5257e-05,  ..., 2.5257e-05,
          2.5257e-05, 2.5257e-05],
         [2.5257e-05, 2.5257e-05, 2.5257e-05,  ..., 2.5257e-05,
          2.5257e-05, 2.5257e-05],
         [2.5257e-05, 2.5257e-05, 2.5257e-05,  ..., 2.5257e-05,
          2.5257e-05, 2.5257e-05],
         ...,
         [2.5257e-05, 2.5257e-05, 2.

In [57]:
torch.sum(result_2-result)

tensor(0.)

In [21]:
print(sum)


tensor([[[2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176],
         [2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176],
         [2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176],
         ...,
         [2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176],
         [2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176],
         [2.8176, 2.8176, 2.8176,  ..., 2.8176, 2.8176, 2.8176]],

        [[2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920],
         [2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920],
         [2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920],
         ...,
         [2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920],
         [2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920],
         [2.5920, 2.5920, 2.5920,  ..., 2.5920, 2.5920, 2.5920]],

        [[2.3387, 2.3387, 2.3387,  ..., 2.3387, 2.3387, 2.3387],
         [2.3387, 2.3387, 2.3387,  ..., 2.3387, 2.3387, 2.3387],
         [2.3387, 2.3387, 2.3387,  ..., 2.3387, 2.3387, 2.

In [8]:
result

tensor(indices=tensor([[     0,      0,      0,  ..., 150527, 150527, 150527],
                       [     0,      1,      2,  ...,     61,     62,     63],
                       [     0,      0,      0,  ...,    221,    221,    221],
                       [     0,      0,      0,  ...,    221,    221,    221]]),
       values=tensor([-0.1496, -0.0656,  0.1445,  ..., -0.1540,  0.1801,
                      -0.0743]),
       size=(150528, 64, 222, 222), nnz=85162752, layout=torch.sparse_coo)

In [15]:
dense = sparse_tensor.to_dense()

In [ ]:
result2 = function(dense)

RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[9075, 128, 51, 51] to have 64 channels, but got 128 channels instead

In [64]:

test = SparseEvaluation(result,function=function2,chunk_size=100)
with torch.no_grad():
    result_3,sum_3 = test.evaluate_all_chunks()


100%|██████████| 91/91 [00:12<00:00,  7.44it/s]


Somme des convolutions sur la dimension 0:
tensor([[[2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05],
         [2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05],
         [2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05],
         ...,
         [2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05],
         [2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05],
         [2.4904e-05, 2.4904e-05, 2.4904e-05,  ..., 2.4904e-05,
          2.4904e-05, 2.4904e-05]],

        [[2.2539e-05, 2.2539e-05, 2.2539e-05,  ..., 2.2539e-05,
          2.2539e-05, 2.2539e-05],
         [2.2539e-05, 2.2539e-05, 2.2539e-05,  ..., 2.2539e-05,
          2.2539e-05, 2.2539e-05],
         [2.2539e-05, 2.2539e-05, 2.2539e-05,  ..., 2.2539e-05,
          2.2539e-05, 2.2539e-05],
         ...,
         [2.2539e-05, 2.2539e-05, 2.

In [65]:
result_4 = function2(result_3.to_dense())

RuntimeError: Given groups=1, weight of size [128, 64, 3, 3], expected input[9075, 128, 51, 51] to have 64 channels, but got 128 channels instead

In [10]:

test = SparseEvaluation(result,function=function2,chunk_size=100)
with torch.no_grad():
    result = test.evaluate_all_chunks()
print(result)

100%|██████████| 1506/1506 [1:02:23<00:00,  2.49s/it]


Somme des convolutions sur la dimension 0:
tensor([[[2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933],
         [2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933],
         [2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933],
         ...,
         [2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933],
         [2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933],
         [2.2933, 2.2933, 2.2933,  ..., 2.2933, 2.2933, 2.2933]],

        [[2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230],
         [2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230],
         [2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230],
         ...,
         [2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230],
         [2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230],
         [2.5230, 2.5230, 2.5230,  ..., 2.5230, 2.5230, 2.5230]],

        [[2.0997, 2.0997, 2.0997,  ..., 2.0997, 2.0997, 2.0997],
         [2.0997, 2.0997, 2.0997,  ..., 2.0997, 2.0997, 2.0997],
         [2.099

In [12]:
import torch
import torch.nn.functional as F
from torch.sparse import FloatTensor
from typing import Callable
from dask.distributed import Client, as_completed
import dask
import dask.array as da
import numpy as np
from tqdm import tqdm

device = torch.device("cpu")

class SparseEvaluation:
    def __init__(self, x: FloatTensor, function: Callable, chunk_size: int):
        self.x = x
        self.function = function
        self.chunk_size = chunk_size
        self.dense_shape = list(x.size())

        # Nombre de chunks
        self.num_chunks = (self.dense_shape[0] + self.chunk_size - 1) // self.chunk_size

        # Taille de sortie de la fonction
        x0 = torch.zeros(1, self.dense_shape[1], self.dense_shape[2], self.dense_shape[3]).to(device)
        self.output_size = list(self.function(x0).shape)
        self.output_size[0] = self.dense_shape[0]

        self.indices = self.x.indices()
        self.values = self.x.values()

        self.indices = self.indices.t()
        self.function_sum = None
        self.global_storage = {
            'indices': [],
            'values': []
        }

    def process_chunk(self, chunk_start, chunk_end):
        with torch.no_grad():
            mask = (self.indices[:, 0] >= chunk_start) & (self.indices[:, 0] < chunk_end)

            chunk_indices = self.indices[mask]
            chunk_indices[:, 0] -= chunk_start

            chunk_values = self.values[mask]
            chunk_size = chunk_end - chunk_start

            chunk_sparse_tensor = torch.sparse.FloatTensor(
                chunk_indices.t(), chunk_values,
                torch.Size([chunk_size] + self.dense_shape[1:])
            )

            chunk_dense_tensor = chunk_sparse_tensor.to_dense().to(device)
            func_output = self.function(chunk_dense_tensor)

            func_sum = torch.abs(func_output).sum(dim=0)

            func_output_sparse = func_output.to_sparse()
            add_indices = func_output_sparse.indices().to(torch.int32) + torch.tensor(
                [[chunk_start], [0], [0], [0]], dtype=torch.int32, device=device
            )

            return func_sum.cpu(), add_indices.cpu(), func_output_sparse.values().cpu()

    def evaluate_all_chunks(self):
        with torch.no_grad():
            client = Client()  # Dask client, can be pointed to a scheduler address

            futures = []
            for i in range(self.num_chunks):
                chunk_start = i * self.chunk_size
                chunk_end = min((i + 1) * self.chunk_size, self.dense_shape[0])
                futures.append(client.submit(self.process_chunk, chunk_start, chunk_end))

            for future in tqdm(as_completed(futures), total=len(futures)):
                func_sum, add_indices, func_output_values = future.result()

                if self.function_sum is None:
                    self.function_sum = func_sum
                else:
                    self.function_sum += func_sum

                self.global_storage['indices'].append(add_indices)
                self.global_storage['values'].append(func_output_values)

            # Concaténation finale des indices et valeurs
            global_indices = torch.cat(self.global_storage['indices'], dim=1)
            global_values = torch.cat(self.global_storage['values'], dim=0)

            global_sparse_tensor = torch.sparse_coo_tensor(global_indices, global_values, size=self.output_size)

            print("Somme des convolutions sur la dimension 0:")
            print(self.function_sum)

            print("\nTenseur sparse global:")
            return global_sparse_tensor,function_sum

# Example usage:
# Assuming `input_tensor` is your sparse input tensor and `your_function` is the function to apply
# sparse_eval = SparseEvaluation(input_tensor, your_function, chunk_size=32)
# result = sparse_eval.evaluate_all_chunks()


ImportError: dask.distributed is not installed.

Please either conda or pip install distributed:

  conda install dask distributed             # either conda install
  python -m pip install "dask[distributed]" --upgrade    # or pip install